# Simulazione modello AIM

In [1]:
import numpy as np
import numba
from numba import njit, prange
from scipy.io import savemat
import matplotlib.pyplot as plt
from simula1d import *

Ho un po' di funzioni che fanno evoluzione temporale, scrivo qua sotto per non dimenticare:
- evoluzione_profili = profili e basta solo alla fine (non ha senso usarla)
- evoluzione_filmino = salva densità e magnetizzazione che sono effettivamente da controllare, secondo quello che dice array
- evoluzione_track = tiene traccia casualmente di alcune particelle per osservare meglio la dinamica
- evoluzione_scorporata = scorpora il meccanismo di flip da quello della diffusione

Compilazioni preliminari per velocizzare

In [2]:
L = 5
n = 200
particles = np.empty((n,2), dtype=np.int64)
particles[:,0] = np.random.randint(0, L, size=n)
particles[:,1] = 2*np.random.randint(0,2,size=n) - 1

density = np.zeros(L, np.int32)
magnet  = np.zeros(L, np.int32)

# popola matrici iniziali
for k in range(n):
    x, s = particles[k]
    density[x] += 1
    magnet[x]  += s

array = np.arange(0,0)
rateflip_numba(L,1,1,1,1,1,magnet,density)
evoluzione_filmino(L,1,1,1,particles,10,1,1,array) #compila

(array([], shape=(0, 5), dtype=float64),
 array([], shape=(0, 5), dtype=float64),
 array([0.14, 0.28, 0.22, 0.2 , 0.11, 0.09, 0.01, 0.05, 0.01, 0.16]))

In [ ]:
# Parametri delle simulazioni
L = 1000
D, epsilon  = 1.0, 0.9
t_max       = 1_000_000
freq_frames = 2_500
log_array = build_log_indices(t_max, freq_frames)
num = 21 #numero di simulazioni da lanciare per ogni taglia

T         = np.array([0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2]) #temperature
betas     = 1/T
gammas    = np.array([0.09, 0.10, 0.11, 0.12, 0.13, 0.14, 0.15]) # grado di località
metodi    = np.array([3, 3, 3, 3, 3, 3, 3]) # smetodo di "località"


from scipy.io import loadmat
percorso_magnet = '/Users/annat/Dati/Tempo formazione aster 1D/1Dtime2T0.200L2000localframes_m.mat'
percorso_density = '/Users/annat/Dati/Tempo formazione aster 1D/1Dtime2T0.200L2000localframes_d.mat'

dati = loadmat(percorso_magnet)
dati2 = loadmat(percorso_density)

magnet = dati['frames_m']
magnet = np.array(magnet, dtype=np.float64)

density = dati2['frames_d']
density = np.array(density, dtype=np.float64)

magnet_asters = magnet[-1, :]
magnet_asters =  np.squeeze(magnet_asters)

density_asters = density[-1, :]
density_asters =  np.squeeze(density_asters)


for j in range(1, num):
    for i, beta in enumerate(betas):
    
        # chiama la funzione numba (già compilata)
        frames_d, frames_m = evoluzione_destroy(
            magnet_asters, density_asters, D, epsilon,
            beta, t_max, log_array, gammas[i], metodi[i])

        # salvataggio per successiva visualizzazione (anche in Matlab)

        savemat(f'1Ddestroy{j}T{1/betas[i]:.3f}L2000' + f'gamma{gammas[i]:.4f}'+'frames_d'+'.mat', {'frames_d': frames_d})
        savemat(f'1Ddestroy{j}T{1/betas[i]:.3f}L2000' + f'gamma{gammas[i]:.4f}'+'frames_m'+'.mat', {'frames_m': frames_m})